<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH18_05_KoBERT%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_KorNLI_%ED%92%80%EC%96%B4%EB%B3%B4%EA%B8%B0(%EB%8B%A4%EC%A4%91%ED%81%B4%EB%9E%98%EC%8A%A4_%EB%B6%84%EB%A5%98).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* KorNLI 데이터는 카카오브레인에서 공개한 한국어 벤치마크 데이터셋임
* 깃허브: https://github.com/kakaobrain/KorNLUDatasets


NLI(Natural Language Inferencing)는 두개의 문장이 주어지고, 두개의 문장이 수반관계인지, 모순관계인지, 중립관계인지를 맞추는 문제임. 

두개의 문장을 입력받아 위의 세개 관계 중 하나를 맞추는 다중 클래스 분류 문제를 풀어봄. 그리고 BERT가 두개의 문장을 입력받을 때는 어떤식으로 전처리를 진행하는지 찾아봄

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 11.3 MB/s eta 0:00:00


### 1. 데이터 로드 및 정제

In [2]:
import os
import pandas as pd
import numpy as np
import urllib.request
from tqdm import tqdm
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
# 데이터 로드
# 훈련데이터
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/\
KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv", filename="multinli.\
train.ko.tsv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/\
KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv", filename="snli_1.0\
_train.ko.tsv")

# 검증데이터
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/\
KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv", filename="xnli.dev.ko.tsv")
# 테스트 데이터
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/\
KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv", filename="xnli.test.ko.tsv")


('xnli.test.ko.tsv', <http.client.HTTPMessage at 0x7f694819a4d0>)

In [4]:
train_snli = pd.read_csv('snli_1.0_train.ko.tsv', sep='\t', quoting=3)
train_xnli = pd.read_csv('multinli.train.ko.tsv', sep='\t', quoting=3)
val_data = pd.read_csv('xnli.dev.ko.tsv', sep='\t', quoting=3)
test_data = pd.read_csv('xnli.test.ko.tsv', sep='\t', quoting=3)

In [5]:
train_snli.shape, train_xnli.shape

((550152, 3), (392702, 3))

In [6]:
# 훈련데이터로 사용할 두개의 데이터프레임을 학습을 위해 합치기 ==> shuffle
train_data = train_snli.append(train_xnli)
train_data = train_data.sample(frac=1)  # shuffle

<ipython-input-6-a4750581e2bb>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_snli.append(train_xnli)


In [7]:
train_data.head()

,sentence1,sentence2,gold_label
322676,아버지와 아들은 아들이 기본적인 스노클 기술을 가르치기 때문에 바위가 많은 해안을 ...,사람들은 콘크리트 위에 있었다,contradiction
169808,세 명의 하키 선수가 골문 앞에서 퍽과 싸운다.,개가 음식을 먹고 있다,contradiction
191720,"그래, 음, 난 내 집에서 사업을 하고 있어.",나는 가정 사업가이다.,entailment
319463,"그리고 오디오 테이프도 있고, 그들은 그들이 정말로 재판에 회부되어서는 안 될 정도...",오디오 테이프는 정말 분명하고 그들이 사용하는 주요 증거물이다.,neutral
207991,"뭐, 뭔가 바뀌어야 할 것 같은데 의회를 보면 얼마나 많은 사람들이 재선되는지 알 ...",우리는 정부에 새로운 사람들을 구해야 한다.,neutral


In [8]:
val_data.head()

,sentence1,sentence2,gold_label
0,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.,neutral
1,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 한마디도 하지 않았다.,contradiction
2,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 엄마에게 집에 갔다고 말했다.,entailment
3,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,나는 워싱턴에 가본 적이 없어서 거기 배정을 받았을 때 그 장소를 찾으려다가 길을 ...,neutral
4,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,워싱턴으로 진군하면서 해야 할 일이 무엇인지 정확히 알고 있었다.,contradiction


In [9]:
test_data.head()

,sentence1,sentence2,gold_label
0,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 그와 다시 이야기하지 않았다.,contradiction
1,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 다시 그와 이야기를 하기 시작했다는 것에 너무 화가 났다.,entailment
2,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",우리는 좋은 대화를 나눴다.,neutral
3,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",그날 현장에 나만 있었던 게 아니라는 걸 몰랐던 것이다.,neutral
4,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",나는 AFFC 공군 경력 분야에서 그 번호를 가진 유일한 사람이라는 인상을 가지고 ...,entailment


In [10]:
# 중복, 결측값 제거
def drop_na_and_duplicates(df):
  df = df.dropna()
  df = df.drop_duplicates()
  df = df.reset_index(drop=True)
  return df

In [11]:
train_data = drop_na_and_duplicates(train_data)
val_data = drop_na_and_duplicates(val_data)
test_data = drop_na_and_duplicates(test_data)

In [12]:
train_data.shape, val_data.shape, test_data.shape

((941814, 3), (2490, 3), (5010, 3))

### 2. BERT 입력

In [13]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [14]:
# 첫번째 샘플에 대한 전처리 결과 테스트
max_seq_len = 128
sent1 = train_data['sentence1'].iloc[0]
sent2 = train_data['sentence2'].iloc[0]

print(sent1)
print(sent2)

아버지와 아들은 아들이 기본적인 스노클 기술을 가르치기 때문에 바위가 많은 해안을 탐험한다.
사람들은 콘크리트 위에 있었다


* encoder_plus는 정수 인코딩 결과인 input_ids, 두개의 문장을 구분하기 위한 token_type_ids, 어텐션 마스크에 해당하는 attention_mask를 자동으로 생성하여 리턴해줌. 두개의 문장을 BERT에 넣을 때는 한번에 넣어야 하기 때문에 encoder_plus 입력으로 sent1과 sent2를 이어 넣어줌

In [15]:
encoding_result = tokenizer.encode_plus(sent1, sent2, max_length=max_seq_len, padding='max_length')

중간에 정수 3이 두번 등장함. 그리고 두번째 3 뒤에는 0이 이어짐. 2는 [CLS], 3은 [SEP], 0은 [PAD]를 의미함

In [16]:
print(encoding_result.input_ids)

[2, 4096, 2522, 4200, 2073, 4200, 2052, 4116, 31221, 13249, 2583, 3726, 2069, 7110, 2015, 3624, 2170, 7076, 2116, 1039, 2073, 7960, 2069, 14013, 4538, 18, 3, 3611, 2031, 2073, 12875, 1485, 2170, 1513, 2359, 2062, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


• [CLS] 문장 1 [SEP] 문장 2 [SEP] [PAD] … 중략 … [PAD]

In [17]:
print(encoding_result.token_type_ids)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


문장 1의 구간까지는 0이 반복되다가, 문장2의 구간부터는 1이 반복된 후 문장 2가 끝나면 다시 0이 반복되는 구조임

In [18]:
print(encoding_result.attention_mask)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


실제 단어 토큰위체에는 1이 반복되다가 패딩 토큰인 [PAD]가 등장하는 구간부터는 0이 반복됨

In [19]:
max_seq_len = 128
# 모든 데이터 인코딩 처리
def convert_examples_to_features(sent_list1, sent_list2, max_seq_len, tokenizer):
  input_ids, attention_masks, token_type_ids = [], [], []

  for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total=len(sent_list1)):
    encoding_result = tokenizer.encode_plus(sent1, sent2, max_length=max_seq_len, pad_to_max_length=True)
    input_ids.append(encoding_result['input_ids'])
    attention_masks.append(encoding_result['attention_mask'])
    token_type_ids.append(encoding_result['token_type_ids'])

  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)

  return (input_ids, attention_masks, token_type_ids)

In [20]:
# 훈련데이터 전처리
X_train = convert_examples_to_features(train_data['sentence1'], train_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|          | 0/941814 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
  1%|          | 10188/941814 [00:16<10:35, 1466.08it/s]B

In [21]:
# 첫번째 샘플에 대한 결과 확인
input_id = X_train[0][0]
attention_mask = X_train[1][0]
token_type_id = X_train[2][0]

print('단어에 대한 정수 인코딩 :', input_id)
print('어텐션 마스크 : ', attention_mask)
print('세그먼트 인코딩:', token_type_id)
print('각 인코딩 길이:', len(input_id))
print('정수 인코딩 복원:', tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    2  4096  2522  4200  2073  4200  2052  4116 31221 13249  2583  3726
  2069  7110  2015  3624  2170  7076  2116  1039  2073  7960  2069 14013
  4538    18     3  3611  2031  2073 12875  1485  2170  1513  2359  2062
     3     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
어텐션 마스크 :  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [22]:
# 검증데이터와 테스트데이터도 모두 전처리 수행
X_val = convert_examples_to_features(val_data['sentence1'], val_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)
X_test = convert_examples_to_features(test_data['sentence1'], test_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 5010/5010 [00:02<00:00, 1970.36it/s]


In [23]:
# 레이블도 인코딩 수행
train_label = train_data['gold_label'].tolist()
val_label = val_data['gold_label'].tolist()
test_label = test_data['gold_label'].tolist()

idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

# 고유한 정수로 변환
y_train = idx_encode.transform(train_label)
y_val = idx_encode.transform(val_label)
y_test = idx_encode.transform(test_label)

In [24]:
idx_encode.classes_, idx_encode.transform(list(idx_encode.classes_))

(array(['contradiction', 'entailment', 'neutral'], dtype='<U13'),
 array([0, 1, 2]))

In [25]:
label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
idx_label = {value : key for key, value in label_idx.items()}
print(label_idx)
print(idx_label)

{'contradiction': 0, 'entailment': 1, 'neutral': 2}
{0: 'contradiction', 1: 'entailment', 2: 'neutral'}


In [26]:
print('변환 전:', train_label[:5])
print('변환 후:', y_train[:5])

변환 전: ['contradiction', 'contradiction', 'entailment', 'neutral', 'neutral']
변환 후: [0 0 1 2 2]


### 3. BERT를 이용한 다중 클래스 분류 모델
다중 클래스 분류 문제를 수행하는 모델로, BERT의 CLS 토큰의 출력에 소프트맥스 함수를 사용하는 출력층을 연결함.

BERT의 출력을 outputs로 리턴받았다면, outputs[1]을 통해 CLS 토큰위치의 벡터를 사용할 수 있음

In [35]:
class TFBertForSequenceClassification(tf.keras.Model):
  def __init__(self, model_name, num_labels):
    super(TFBertForSequenceClassification, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
    self.classifier = tf.keras.layers.Dense(num_labels, 
                                            kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                            activation='softmax', name='classifier')
  
  def call(self, inputs):
    input_ids, attention_mask, token_type_ids = inputs
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    cls_token = outputs[1]
    prediction = self.classifier(cls_token)
    return prediction

In [36]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="grpc://" + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [37]:
with strategy.scope():
  model = TFBertForSequenceClassification('klue/bert-base', num_labels=3)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [38]:
# 두번의 에포크 수행
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=2
)

model.fit(X_train, y_train, epochs=2, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/2
29432/29432 [==============================] - 2472s 81ms/step - loss: 0.5337 - accuracy: 0.7876 - val_loss: 0.5585 - val_accuracy: 0.7779
Epoch 2/2
29432/29432 [==============================] - 2306s 78ms/step - loss: 0.4350 - accuracy: 0.8329 - val_loss: 0.5543 - val_accuracy: 0.7847


In [39]:
# 테스트 데이터 정확도 확인
print("\n 테스트 정확도 : %.4f" % (model.evaluate(X_test, y_test, batch_size=1024)[1]))

5/5 [==============================] - 14s 2s/step - loss: 0.5377 - accuracy: 0.7862

 테스트 정확도 : 0.7862


### 4. TFBertForSequenceClassification 구현

In [40]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model = TFBertForSequenceClassification.from_pretrained('klue/bert-base', num_labels=3, from_pt=True)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
